# Part 1 Coursework


# Desmond Ojei  1419100

### <u>Importing packages below<u>:

In [1]:

#Importing all packages needed to carry out the project

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from pathlib import Path
from PIL import Image
from skimage import feature
import imgaug as imaug
import imgaug.augmenters 
from sklearn.cluster import KMeans
import math


C:\Users\hp\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
#Checking for the working directory
os.getcwd()


'C:\\Users\\hp\\Desktop\\Data Science\\Second Semester\\Advanced Data Science'

### <u>Preprocessing Image<u>
The following steps carried out be carried out on the image:
* step 1 -> Loading Images
* step 2 -> Image Re-sizing 
* step 3 -> Image Normalisation 
* step 4 -> Image Binarization  
* step 5 -> image Flattening 
* step 6 -> Creating a class for images 

### <u>Justification of techniques <u>
Data preprocessing is a technique which involves cleaning raw data into an understandable and useful format.However,after loading the images by importing "Image" from the package "PIL", this creates a PIL Image object,which  was converted to an array as seen on the code below, as the images cant be read when not in an array. Also, in other to remove channels from the images an ensure uniformity of image colours,the ".convert("L")" was added.  
Furthermore, for the algorithm to be successful, the images need to a be processessed in a uniform format across all images.This implies that images loaded should look the same,due to the fact that some images might be lighter than the others, it is best to convert all images to a greyscale, also the size of all images should be the same, in the code below the size is set to  (100*100).In addition, normalizing the image dataset ensures the data set is set to a common scale, without any distortion differences in the value ranges. Also, binary threshold is useful when applying feature engineering which helps improve machine learning.Finally, flattening an image significantly decreases size of file, significantly making the image easier to print and export.


In [3]:
# First setting the directory to read the Symbols folder
img_dir = ("C:\\Users\\hp\\Desktop\\Data Science\\Second Semester\\Advanced Data Science")

#Creating an empty list
images=[]
class_of_images =[]

#using the Pathlib package to import Path
# the for loop basically iterates over every path in the the set directory and sub_directories find files with "jpg"
#for each image in the symbols folders this code applies all the pre-processing steps as stated above

for path in list(Path(img_dir).rglob('*.jpg'))[:]:

   #print(path)
    #print(path.name)
    #("step 1 -> Loading images")
    im = Image.open(path)
    
    #Converting image to grayu scale in other to remove channels
    im_to_greyscale = im.convert("L")
    
    #converting image to an array 
    pixel_array = np.array(im_to_greyscale)
    
    #"step 2 ->Resizing images" 
    resize_image = cv2.resize(pixel_array,(50,50))
    #print("step 2 -> resized shape :" + str(resize_image.shape) )
   
    #("step 3 ->Normalising image")
    Normalized_image =  resize_image/255
    ##print(Normalized_image)
    

    #("step 4 ->Binarisation")
    #Binarization 
    _,image_thresh1 = cv2.threshold(Normalized_image,0.5,1,cv2.THRESH_BINARY)
    
    
    #("step 5 ->Flattened")
   #Flattening images with the Normalised image
    flattened_images =  image_thresh1.flatten()
 
    
    #adding that list to the empty list cretaed above the for the loop
    images.append(flattened_images)
    
    classname = os.path.dirname(path).split("\\")[-1]

    #Creating class for images
    class_of_images.append(classname)
    
images = np.array(images)
print(images.shape)
print(images)

print("step 6 -> Classes: " + str(class_of_images))
# #Converting the images into an array
# images = np.array(images)
# print(images)


    
    


    
    



(2432, 2500)
[[0. 0. 0. ... 1. 1. 1.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
step 6 -> Classes: ['Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arrowhead', 'Arro

### Feature Extraction:
* HOG (Histogram Oriented Gradient)  
The  Histogram  Oriented Gradients, is a feature extraction technique which is used to extract features from an image data. This technique is used to perform  computer vision tasks.In addtion HOG has the ability to carry out the edge direction as well which is achieved by extracting the gradient and orientation of the edges from the image.

In [4]:
#Using the HOG (Histogram Oriented Gradient)     
class HOG:
    def __init__(self, orientations = 9, pixelsPerCell = (8, 8),
        cellsPerBlock = (3, 3), transform = False):
        # store the number of orientations, pixels per cell,
        # cells per block, and whether or not power law
        # compression should be applied
        self.orienations = orientations
        self.pixelsPerCell = pixelsPerCell
        self.cellsPerBlock = cellsPerBlock
        self.transform = transform

    def describe(self, image):
        # compute HOG for the image
        hist = feature.hog(image, orientations = self.orienations,
            pixels_per_cell = self.pixelsPerCell,
            cells_per_block = self.cellsPerBlock,
            transform_sqrt = self.transform)
        ## return the HOG features
        return hist
    
hog = HOG(orientations = 18, pixelsPerCell = (10, 10), cellsPerBlock = (1, 1), transform = True)
    
# print("PLot of image")

img_dir = ("C:\\Users\\hp\\Desktop\\Data Science\\Second Semester\\Advanced Data Science\\Symbols")
#Creating an empty list
bin_thresh = 180
image_symbols = [] # List to append the symbols as 2D numpy arrays. This is useful to extract features afterwards
class_image_fx = [] # List to append the target
hogrepo = []  # Create a list to append the HOG features


 #using the Pathlib package to import Path
 # the for loop basically iterates over every path in the the set directory and sub_directories find files with "jpg"

for path in Path(img_dir).rglob('*.jpg'):
     #("step 1 -> Loading images")
    image = Image.open(path)
     #converting image to an array 
    image = np.array(image)

    #"step 2 ->Resizing images"
    image = cv2.resize(image,(50,50))
   
    #("step 4 ->Binarisation")
    #Binarization 
    _,image = cv2.threshold(image, bin_thresh, 255, 0) 
    image_symbols.append(image)
   
    # Extract HOG and append to HOG repo
    hist = hog.describe(image)
    
    hogrepo.append(hist)
            
    classname= os.path.dirname(path).split("\\")[-1]
    class_image_fx.append(classname)   
#Convert the three repo lists into numpy arrays
 
hogrepo = np.array(hogrepo) # convert into numpy array
print(hogrepo.shape)
print(hogrepo) 



C:\Users\hp\Anaconda3\lib\site-packages\skimage\feature\_hog.py:150: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15. To supress this message specify explicitly the normalization method.
  skimage_deprecation)


(2432, 450)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.53416396 0.         0.         ... 0.         0.         0.        ]
 [0.24205004 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


### <u>Image Augmentation<u>
There are several ways to carry out image augementation in image processing before it is passed through a classifier using python. This process can be achieve by using Keras,Tensorflow etc. However, in this project , the  library "imgaug" was installed. This library contains alof of augmentation techniques in the form of classes which is derived from the Augmenter parent class. Firstly,Sequential was used to combine several augmentation techniques.The techniques include the following below: 
After perusing through the images in the symbols folder, most images were either rotated zoomed out or zoomed in etc,and there were no images sheared or applied with elastic tranformation, therefore in this project the following augmentation techniques were used in other to improve the classifier even better.

**<u>Affine<u>:** This technique was used to shear the image by 0 to 50 degerees.
    
**<u>Elastic Transformation <u>:** This gives the image a water effect look.





In [6]:
images_augElastic = []

for path in list(Path(img_dir).rglob('*.jpg'))[:4]:
   #print(path)
    #print(path.name)
    #("step 1 -> Loading images")
    im = Image.open(path)
    
    #Converting image to grayu scale in other to remove channels
    im_to_greyscale = im.convert("L")
    
    #converting image to an array 
    pixel_array = np.array(im_to_greyscale)
    
    #"step 2 ->Resizing images" 
    resize_image = cv2.resize(pixel_array,(50,50))
    #print("step 2 -> resized shape :" + str(resize_image.shape) )
   
    #("step 3 ->Normalising image")
    Normalized_image =  resize_image/255
    ##print(Normalized_image)
    

    #("step 4 ->Binarisation")
    #Binarization 
    _,image_thresh1 = cv2.threshold(Normalized_image,0.5,1,cv2.THRESH_BINARY)
    #print(image_thresh1)
    
    images_augElastic.append(image_thresh1)
    
#This gives a water effect on images
    elastic=imgaug.augmenters.ElasticTransformation(alpha=90, sigma=9)
    elastic_image= elastic.augment_images(images_augElastic)
print(elastic_image)
    


  
    

[array([[-0.00274658,  0.        , -0.03515625, ..., -0.11103058,
        -0.07177734, -0.01244123],
       [ 0.25914002,  0.10449219,  0.60505674, ...,  0.57019041,
         0.80528261,  0.05224609],
       [ 1.10986325,  1.1001358 ,  1.05815888, ...,  0.99999997,
         1.00000003,  0.64620733],
       ...,
       [ 1.        ,  0.99999999,  0.99999999, ...,  1.00000004,
         1.        ,  1.00000003],
       [ 1.        ,  1.00000001,  0.99999999, ...,  1.00597383,
         1.00000003,  1.00000002],
       [ 1.00000003,  1.00000003,  1.        , ...,  1.08342743,
         0.99999997,  0.99999997]]), array([[ 1.11077881,  1.02142332,  0.99999999, ...,  1.028038  ,
         0.86570501,  0.        ],
       [ 1.03515625,  1.00000001,  1.        , ...,  1.09375   ,
        -0.02199554,  0.        ],
       [ 0.99999997,  1.00000003,  1.00000001, ...,  1.10741426,
        -0.07177734,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        

In [7]:

images_flip =[]
for path in list(Path(img_dir).rglob('*.jpg'))[:5]:
   #print(path)
    #print(path.name)
    #("step 1 -> Loading images")
    im = Image.open(path)
    
    #Converting image to grayu scale in other to remove channels
    im_to_greyscale = im.convert("L")
    
    #converting image to an array 
    pixel_array = np.array(im_to_greyscale)
    
    #"step 2 ->Resizing images" 
    resize_image = cv2.resize(pixel_array,(50,50))
    #print("step 2 -> resized shape :" + str(resize_image.shape) )
   
    #("step 3 ->Normalising image")
    Normalized_image =  resize_image/255
    ##print(Normalized_image)
    

    #("step 4 ->Binarisation")
    #Binarization 
    _,image_thresh1 = cv2.threshold(Normalized_image,0.5,1,cv2.THRESH_BINARY)
    #print(image_thresh1)
    
    #print("step 5 ->Flattened")
   #Flattening images with the Normalised image
    flattened_images =  image_thresh1.flatten()
    #print(flattened_images, flattened_images.size)
    
    #adding that list to the empty list cretaed above the for the loop
    images_flip .append(image_thresh1)
    
    #This gives a water effect on images
    
    #Shearing the image by 0 to 50 degrees
    flip = imgaug.augmenters.Flipud(p=(0.5))
    flip_image=flip.augment_images(images_flip)
print(flip_image)



[array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]]), array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 1., 1.]]), array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]]), array([[1., 1., 1., ..., 0., 0.

### Class Decomposition 


Class decomposition is the process of segmenting each class in a dataset into a number of homogeneous subclasses.In this project, I used class decomposition to increase  diversity because this  a key factor for the success of ensemble classifiers.
Also, this process is known to be  computationally efficient method to provide a linearly separable data set without the need for feature engineering required by techniques such as deep learning etc. A seen below In other to achieve a balance ie IR ~ 1, I used the standard balancing equation: ki = [ ci/ Avg+ 1]

* ki is the number of clusters.
* ci is the total number of instances of a specific class.
* Avg is the mean class distribution

In [8]:
#First we calculating Avg
##Then we can calculate ki for each class
frequency_class=[]
unique_class = np.unique(np.array(class_image_fx)) 
unique_class.sort()
Avg = len(class_image_fx)/len(unique_class)
for name in unique_class:
    frequency_class.append(class_image_fx.count(name))
print("ki for each class")
for i,c in enumerate(frequency_class): 
     print('k_'+str(i)+'=',(c/Avg)+1)
        
#Since ki is larger than 2, then you floor the obtained value and you get the clusters needed
k_list=[] 
for c in (frequency_class): 
    K_means = math.floor(c/Avg)+1
    k_list.append( K_means)
print("k values for each class:",k_list)

ki for each class
k_0= 4.848684210526316
k_1= 2.331003289473684
k_2= 1.0160361842105263
k_3= 1.1122532894736843
k_4= 2.8601973684210527
k_5= 1.208470394736842
k_6= 1.0801809210526316
k_7= 1.0320723684210527
k_8= 1.3688322368421053
k_9= 3.036595394736842
k_10= 1.6254111842105263
k_11= 2.8120888157894735
k_12= 1.0641447368421053
k_13= 2.0423519736842106
k_14= 1.1122532894736843
k_15= 1.1443256578947367
k_16= 1.1924342105263157
k_17= 1.2726151315789473
k_18= 1.7376644736842106
k_19= 1.9782072368421053
k_20= 1.6895559210526316
k_21= 1.0641447368421053
k_22= 5.5703125
k_23= 1.1603618421052633
k_24= 7.286184210526316
k_25= 1.6735197368421053
k_26= 1.096217105263158
k_27= 2.1866776315789473
k_28= 1.0160361842105263
k_29= 1.6414473684210527
k_30= 1.4329769736842106
k_31= 3.7742598684210527
k_32= 2.138569078947368
k_33= 3.020559210526316
k_34= 1.0641447368421053
k_35= 1.4490131578947367
k_36= 2.411184210526316
k_37= 1.3848684210526316
k_38= 1.0641447368421053
k values for each class: [4, 2, 1, 

In [9]:
def class_decomposition(data, target, k_means):
    target_cd = ['']*len(target)
    print('\nClass decomposition by applying k-means...')    
    for a, label in enumerate(unique_class):
        ## Split the dataset
        X = []
        y = []
        for x, dat in enumerate(data):
            if target[x]==label:
                X.append(dat)
                y.append(x)
        if  1<k_means[a]<=len(X):
                ## Applying k-means to the list    
                kmeans = KMeans(n_clusters=k_means[a], random_state=0).fit(X)
                for n, m in enumerate(kmeans.labels_):
                    target_cd[y[n]]=str(label)+'_c'+str(m)
                             
        else:
            for m in y:
                     target_cd[m]=str(label)+'_c0'
        return  target_cd
target_cd = class_decomposition(hogrepo,class_image_fx, k_list)
print(target_cd)



Class decomposition by applying k-means...
['Arrowhead_c2', 'Arrowhead_c1', 'Arrowhead_c1', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c1', 'Arrowhead_c3', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c3', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c1', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c1', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c1', 'Arrowhead_c2', 'Arrowhead_c2', 'Arrowhead_c3', 'Arrowhead_c3', 'Arrowhead_c3', 'Arrowhead_c3', 'Arrowhead_c3', 'Arrowhead_c3', 'Arrowhead_c3', 'Arrowhead_c3', 'Arrowhead_c1', 'Arrowhead_c3', 'Arrowhead_c2', 'Arrowhead_

# Part 2  Coursework


## RandomForest Classifier

In [10]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from sklearn.metrics import classification_report,confusion_matrix

Using TensorFlow backend.


In [11]:
#Loading up datasets
loading_data_1 = pd.read_csv("Symbols_pixel_cd.csv",header = None)
loading_data_2 =  pd.read_csv("Symbols_pixel.csv",header = None)
loading_data_3 =  pd.read_csv("Symbols_hog.csv",header = None)
loading_data_4 =  pd.read_csv("Symbols_lbp.csv",header = None)
loading_data_5 =  pd.read_csv("Symbols_hog_ros.csv",header = None)

#Changing pandas dataframe to numpy array
X_Symbols_1 = loading_data_1.iloc[:,0:10000].values
y_target_1 = loading_data_1.iloc[:,10000].values

X_Symbols_2 = loading_data_2.iloc[:,0:10000].values
y_target_2 = loading_data_2.iloc[:,10000].values

X_Symbols_3= loading_data_3.iloc[:,0:1800].values
y_target_3 = loading_data_3.iloc[:,1800].values

X_Symbols_4= loading_data_4.iloc[:,0:26].values
y_target_4 = loading_data_4.iloc[:,26].values

X_Symbols_5= loading_data_5.iloc[:,0:1800].values
y_target_5 = loading_data_5.iloc[:,1800].values

print(X_Symbols_1,y_target_1)
print(X_Symbols_2,y_target_2)
print(X_Symbols_3,y_target_3)
print(X_Symbols_4,y_target_4)
print(X_Symbols_5,y_target_5)

[[  0   0   0 ... 255 255 255]
 [255 255 255 ...   0   0   0]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]] ['Arrowhead_c1' 'Arrowhead_c1' 'Arrowhead_c3' ... 'Vessel_c0' 'Vessel_c0'
 'Vessel_c0']
[[  0   0   0 ... 255 255 255]
 [255 255 255 ...   0   0   0]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]] ['Arrowhead' 'Arrowhead' 'Arrowhead' ... 'Vessel' 'Vessel' 'Vessel']
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.50007804 0.         0.         ... 0.         0.         0.        ]
 [0.61134064 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]] ['Arrowhead' 'A

#### Using this cell to split your dataset into training and testing the models


At this point, I trained a supervised learning model known as RandomForest  to evaluate.The aim is to pass 5 datasets into this classifier and compare the performaces and see which dataset the model performs best. 

First the datasets are split  into training and testing sets (70% train, 30% test) 





In [12]:
# Using this cell to split your dataset into training and testing w/ stratification
X_train_1,X_test_1,y_train_1,y_test_1 = train_test_split(X_Symbols_1,y_target_1,test_size=0.3)
X_train_2,X_test_2,y_train_2,y_test_2 = train_test_split(X_Symbols_2,y_target_2,test_size=0.3)
X_train_3,X_test_3,y_train_3,y_test_3 = train_test_split(X_Symbols_3,y_target_3,test_size=0.3)
X_train_4,X_test_4,y_train_4,y_test_4 = train_test_split(X_Symbols_4,y_target_4,test_size=0.3)
X_train_5,X_test_5,y_train_5,y_test_5 = train_test_split(X_Symbols_5,y_target_5,test_size=0.3)

#Normalising
X_train_1=  X_train_1 / 255
X_test_1 = X_test_1/255

X_train_2=  X_train_2 / 255
X_test_2= X_test_2/255


#### Using this cell to train a RandomForest model to predict the labels of the test data
Using the default parameters for the modelS.

In [198]:
#using the default parameters for the model.

print("model_1 : Symbols_pixel_cd.csv")
rf_model_1 = RandomForestClassifier(n_estimators = 1000, random_state = 42)
# Fit the model
clf_rf_1 = rf_model_1.fit(X_train_1, y_train_1)
y_rf_1 = rf_model_1.predict(X_test_1)
print(y_rf_1,y_rf_1.shape)



print("\n")


print("model_2 : Symbols_pixel.csv")
rf_model_2 = RandomForestClassifier(n_estimators = 1000, random_state = 42)
# Fit the model
clf_rf_2 = rf_model_2.fit(X_train_2, y_train_2)
y_rf_2 = rf_model_2.predict(X_test_2)
print(y_rf_2,y_rf_2.shape)


print("\n")


print("model_3 : Symbols_hog.csv")
rf_model_3 = RandomForestClassifier(n_estimators = 1000, random_state = 42)
# Fit the model
clf_rf_3 = rf_model_3.fit(X_train_3, y_train_3)
y_rf_3 = rf_model_3.predict(X_test_3)
print(y_rf_3,y_rf_3.shape)


print("\n")


print("model_4 : Symbols_lbp.csv")
rf_model_4 = RandomForestClassifier(n_estimators = 1000, random_state = 42)
# Fit the model
clf_rf_4 = rf_model_4.fit(X_train_4, y_train_4)
y_rf_4 = rf_model_4.predict(X_test_4)
print(y_rf_4,y_rf_4.shape)


print("\n")


print("model_5 : Symbols_hog_ros.cs")
rf_model_5 = RandomForestClassifier(n_estimators = 1000, random_state = 42)
# Fit the model
clf_rf_5 = rf_model_5.fit(X_train_5, y_train_5)
y_rf_5 = rf_model_5.predict(X_test_5)
print(y_rf_5,y_rf_5.shape)





model_1 : Symbols_pixel_cd.csv
['DB&BBV_c2' 'Arrowhead_c3' 'Continuity Label_c1' 'Spectacle Blind_c0'
 'Sensor_c5' 'Valve Ball_c2' 'Valve Ball_c2' 'Valve Butterfly_c0'
 'Arrowhead_c1' 'Reducer_c1' 'Sensor_c5' 'Arrowhead_c0' 'Sensor_c5'
 'Reducer_c4' 'Reducer_c1' 'DB&BBV + Valve Check_c0' 'DB&BPV_c0'
 'Reducer_c1' 'Spectacle Blind_c0' 'DB&BBV_c0' 'Valve Plug_c1' 'Sensor_c5'
 'DB&BBV_c0' 'Flange Single T-Shape_c0' 'Sensor_c6' 'DB&BPV_c0'
 'Injector Point_c0' 'Valve Ball_c1' 'Arrowhead_c3' 'Valve Check_c2'
 'Arrowhead_c3' 'Continuity Label_c0' 'Flange Single T-Shape_c0'
 'Flange + Triangle_c0' 'Valve_c0' 'Sensor_c1' 'Continuity Label_c0'
 'Spectacle Blind_c0' 'DB&BBV_c0' 'Reducer_c3' 'Triangle_c1'
 'Spectacle Blind_c0' 'Arrowhead + Triangle_c1' 'Reducer_c4'
 'ESDV Valve Ball_c0' 'Sensor_c3' 'Valve Check_c2' 'Arrowhead_c1'
 'Valve Ball_c2' 'ESDV Valve Ball_c1' 'Reducer_c1' 'Reducer_c1' 'Valve_c0'
 'Reducer_c1' 'Reducer_c0' 'ESDV Valve Ball_c1' 'Valve Butterfly_c0'
 'Sensor_c1' 'Sensor_c5' 

### Performance Evaluation Metrics

I used  a confusion metrics as seen below , to meaure the performance on each model with the Random Forest classifier. Also, from the classification report shows the  precision,recall,f1-score and support. Finally using the accuracy_score function, the performance of the model on each dataset will be evaluated with accuracy.

In [199]:
random_state = 42


print("model_1 : Symbols_pixel_cd.csv")
print(classification_report(y_test_1,y_rf_1))


print("\n")


print("model_2 : Symbols_pixel.csv")
print(classification_report(y_test_2,y_rf_2))


print("\n")


print("model_3 : Symbols_hog.csv")
print(classification_report(y_test_3,y_rf_3))


print("\n")


print("model_4 : Symbols_lbp.csv")
print(classification_report(y_test_4,y_rf_4))


print("\n")


print("model_5 : Symbols_hog_ros.cs")
print(classification_report(y_test_5,y_rf_5))


model_1 : Symbols_pixel_cd.csv
                               precision    recall  f1-score   support

      Arrowhead + Triangle_c0       0.83      0.83      0.83         6
      Arrowhead + Triangle_c1       0.92      0.85      0.88        13
                 Arrowhead_c0       0.76      0.84      0.80        19
                 Arrowhead_c1       0.80      0.80      0.80        15
                 Arrowhead_c2       0.56      0.71      0.63         7
                 Arrowhead_c3       0.83      0.86      0.84        28
                       Box_c0       1.00      0.50      0.67         2
          Continuity Label_c0       1.00      0.94      0.97        34
          Continuity Label_c1       0.50      1.00      0.67         1
 Control Valve Angle Choke_c0       0.00      0.00      0.00         1
       Control Valve Globe_c0       0.67      0.25      0.36         8
             Control Valve_c0       0.50      0.33      0.40         3
                   Control_c0       1.00     

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                            precision    recall  f1-score   support

                 Arrowhead       0.99      1.00      1.00       132
      Arrowhead + Triangle       1.00      1.00      1.00       118
                Barred Tee       1.00      1.00      1.00       120
                       Box       1.00      1.00      1.00       133
          Continuity Label       1.00      1.00      1.00       115
                   Control       1.00      1.00      1.00       102
             Control Valve       1.00      1.00      1.00       109
 Control Valve Angle Choke       1.00      1.00      1.00       116
       Control Valve Globe       1.00      1.00      1.00       117
                    DB&BBV       1.00      1.00      1.00       115
      DB&BBV + Valve Check       1.00      1.00      1.00       114
                    DB&BPV       1.00      1.00      1.00       111
                    Deluge       1.00      1.00      1.00       138
           ESDV Valve Ball       1.00      1.00

In [200]:
random_state = 42
A1 = accuracy_score(y_rf_1,y_test_1)
A2 = accuracy_score(y_rf_2,y_test_2)
A3= accuracy_score(y_rf_3,y_test_3)
A4 = accuracy_score(y_rf_4,y_test_4)
A5 = accuracy_score(y_rf_5,y_test_5)
print('Symbols_pixel_cd:', A1*100)
print('Symbols_pixel:', A2*100)
print('Symbols_hog:', A3*100)
print('Symbols_lbp:', A4*100)
print('Symbols_hog_ros:', A5*100)

Symbols_pixel_cd: 83.6986301369863
Symbols_pixel: 92.6027397260274
Symbols_hog: 93.42465753424658
Symbols_lbp: 97.3972602739726
Symbols_hog_ros: 99.97819925877481


## Neural Networks

#### Data Preprocessing

In [21]:
#Changing pandas dataframe to numpy array
X_Symbols_1nn= loading_data_1.iloc[:,0:10000].values
y_target_1nn = loading_data_1.iloc[:,10000].values

X_Symbols_2nn = loading_data_2.iloc[:,0:10000].values
y_target_2nn = loading_data_2.iloc[:,10000].values

X_Symbols_3nn= loading_data_3.iloc[:,0:1800].values
y_target_3nn = loading_data_3.iloc[:,1800].values

X_Symbols_4nn= loading_data_4.iloc[:,0:26].values
y_target_4nn = loading_data_4.iloc[:,26].values

X_Symbols_5nn= loading_data_5.iloc[:,0:1800].values
y_target_5nn = loading_data_5.iloc[:,1800].values


First the datasets are split into training and testing sets (70% train, 30% test)

In [22]:
#Splitting data
random_state = 42
X_train_1nn,X_test_1nn,y_train_1nn,y_test_1nn = train_test_split(X_Symbols_1nn,y_target_1nn,test_size=0.3)
X_train_2nn,X_test_2nn,y_train_2nn,y_test_2nn = train_test_split(X_Symbols_2nn,y_target_2nn,test_size=0.3)
X_train_3nn,X_test_3nn,y_train_3nn,y_test_3nn = train_test_split(X_Symbols_3nn,y_target_3nn,test_size=0.3)
X_train_4nn,X_test_4nn,y_train_4nn,y_test_4nn = train_test_split(X_Symbols_4nn,y_target_4nn,test_size=0.3)
X_train_5nn,X_test_5nn,y_train_5nn,y_test_5nn = train_test_split(X_Symbols_5nn,y_target_5nn,test_size=0.3)




In [23]:
#Normalising
X_train_1nn=  X_train_1nn / 255
X_test_1nn = X_test_1nn/255

X_train_2nn=  X_train_2nn / 255
X_test_2nn = X_test_2nn/255


# print(X_train_1nn)
# print(X_train_2nn)

Using the Multi-Layer Perceptron Classifier model from the neural_network library of SciKit-Learn

In [24]:
from sklearn.neural_network import MLPClassifier
random_state = 42
Neural_Network_1 = MLPClassifier(hidden_layer_sizes=(30,30,30))
Neural_Network_2 = MLPClassifier(hidden_layer_sizes=(30,30,30))
Neural_Network_3 = MLPClassifier(hidden_layer_sizes=(30,30,30))
Neural_Network_4 = MLPClassifier(hidden_layer_sizes=(30,30,30))
Neural_Network_5= MLPClassifier(hidden_layer_sizes=(30,30,30))

In [25]:
# Fitting the each model 
random_state = 42
Neural_Network_1.fit(X_train_1nn,y_train_1nn)
Neural_Network_2.fit(X_train_2nn,y_train_2nn)
Neural_Network_3.fit(X_train_3nn,y_train_3nn)
Neural_Network_4.fit(X_train_4nn,y_train_4nn)
Neural_Network_5.fit(X_train_5nn,y_train_5nn)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\hp\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(30, 30, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [26]:
#Prediction
predictions_1= Neural_Network_1.predict(X_test_1nn)
predictions_2= Neural_Network_2.predict(X_test_2nn)
predictions_3= Neural_Network_3.predict(X_test_3nn)
predictions_4= Neural_Network_4.predict(X_test_4nn)
predictions_5= Neural_Network_5.predict(X_test_5nn)

### Performance Evaluation Metrics

Also, I used the confusion Metrics to meaure the performance on each model with the Neural Network

In [27]:
#Evaluation
from sklearn.metrics import classification_report,confusion_matrix
print("model_1 : Symbols_pixel_cd.csv")
print(classification_report(y_test_1nn,predictions_1))


print("\n")


print("model_2 : Symbols_pixel.csv")
print(classification_report(y_test_2nn,predictions_2))


print("\n")


print("model_3 : Symbols_hog.csv")
print(classification_report(y_test_3nn,predictions_3))


print("\n")


print("model_4 : Symbols_lbp.csv")
print(classification_report(y_test_4nn,predictions_4))


print("\n")


print("model_5 : Symbols_hog_ros.csV")
print(classification_report(y_test_5nn,predictions_5))




model_1 : Symbols_pixel_cd.csv
                               precision    recall  f1-score   support

      Arrowhead + Triangle_c0       1.00      1.00      1.00         3
      Arrowhead + Triangle_c1       0.83      0.71      0.77        21
                 Arrowhead_c0       0.70      0.73      0.71        22
                 Arrowhead_c1       0.67      0.50      0.57        16
                 Arrowhead_c2       0.25      0.20      0.22         5
                 Arrowhead_c3       0.69      0.82      0.75        22
                       Box_c0       0.00      0.00      0.00         1
          Continuity Label_c0       0.97      0.97      0.97        32
          Continuity Label_c1       0.00      0.00      0.00         4
       Control Valve Globe_c0       0.60      0.30      0.40        10
             Control Valve_c0       0.00      0.00      0.00         2
                   Control_c0       1.00      0.67      0.80         3
      DB&BBV + Valve Check_c0       0.75     

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\hp\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


                            precision    recall  f1-score   support

                 Arrowhead       0.99      0.98      0.99       127
      Arrowhead + Triangle       0.99      1.00      1.00       109
                Barred Tee       1.00      1.00      1.00       122
                       Box       1.00      1.00      1.00       121
          Continuity Label       1.00      1.00      1.00       126
                   Control       0.98      1.00      0.99       116
             Control Valve       1.00      1.00      1.00       109
 Control Valve Angle Choke       1.00      1.00      1.00       108
       Control Valve Globe       0.98      1.00      0.99       128
                    DB&BBV       1.00      0.98      0.99       109
      DB&BBV + Valve Check       1.00      1.00      1.00       117
                    DB&BPV       1.00      1.00      1.00       108
                    Deluge       1.00      1.00      1.00       114
           ESDV Valve Ball       0.98      1.00

In [28]:
random_state = 42
A1nn = accuracy_score(predictions_1,y_test_1nn)
A2nn = accuracy_score(predictions_2,y_test_2nn)
A3nn= accuracy_score(predictions_3,y_test_3nn)
A4nn = accuracy_score(predictions_4,y_test_4nn)
A5nn = accuracy_score(predictions_5,y_test_5nn)
print('Symbols_pixel_cd:', A1nn*100)
print('Symbols_pixel:', A2nn*100)
print('Symbols_hog:', A3nn*100)
print('Symbols_lbp:', A4nn*100)
print('Symbols_hog_ros:', A5nn*100)

Symbols_pixel_cd: 76.43835616438356
Symbols_pixel: 80.95890410958904
Symbols_hog: 89.72602739726028
Symbols_lbp: 63.6986301369863
Symbols_hog_ros: 99.73839110529758


   ### Convolutional Neural Network 

#### Data Preprocessing

In [37]:
X_Symbols_1cnn = loading_data_1.iloc[:,0:10000].values
y_target_1cnn = loading_data_1.iloc[:,10000].values

X_Symbols_2cnn = loading_data_2.iloc[:,0:10000].values
y_target_2cnn = loading_data_2.iloc[:,10000].values




70% of the data for training and 30% for testing via the train_test_split  function.

In [38]:
#Splitting data
random_state = 42
X_train_1cnn,X_test_1cnn,y_train_1cnn,y_test_1cnn = train_test_split(X_Symbols_1cnn,y_target_1cnn,test_size=0.3)
X_train_2cnn,X_test_2cnn,y_train_2cnn,y_test_2cnn = train_test_split(X_Symbols_2cnn,y_target_2cnn,test_size=0.3)

In [39]:
print(X_train_1cnn.shape,y_train_1cnn.shape,X_test_1cnn.shape, y_test_1cnn.shape)
print(X_train_2cnn.shape,y_train_2cnn.shape,X_test_2cnn.shape, y_test_2cnn.shape)


(1702, 10000) (1702,) (730, 10000) (730,)
(1702, 10000) (1702,) (730, 10000) (730,)


In [40]:
#As seen, the data is in 3 dimensional and CNN only works with 4 dimension
print(X_train_1cnn.shape)

(1702, 10000)


In [41]:
#Reshaping the data to 4 dimension array
X_train_reshape = X_train_1cnn.reshape(X_train_1cnn.shape[0], 100, 100, 1)
X_test_reshape = X_test_1cnn.reshape(X_test_1cnn.shape[0],100, 100, 1)
print(X_train_reshape.shape)
print(X_test_reshape.shape)

(1702, 100, 100, 1)
(730, 100, 100, 1)


In [42]:
#Reshaping the data to 4 dimension array
X_train_reshape_2 = X_train_2cnn.reshape(X_train_2cnn.shape[0], 100, 100, 1)
X_test_reshape_2 = X_test_2cnn.reshape(X_test_2cnn.shape[0],100, 100, 1)
print(X_train_reshape_2.shape)
print(X_test_reshape_2.shape)

(1702, 100, 100, 1)
(730, 100, 100, 1)


In [43]:
# Convert to float 32
X_train_reshape = X_train_reshape.astype('float32')
X_test_reshape = X_test_reshape .astype('float32')

In [44]:
# Convert to float 32
X_train_reshape_2 = X_train_reshape_2.astype('float32')
X_test_reshape_2 = X_test_reshape_2.astype('float32')
print(len(X_test_reshape_2))

730


In [45]:
X_train_1cnn=  X_train_reshape / 255
X_test_1cnn = X_test_reshape/255



In [46]:
X_train_2cnn=  X_train_reshape_2/ 255
X_test_2cnn = X_test_reshape_2/255
print(len(X_test_2cnn))

730


For instance, if  had 5 classes of data, a one-hot encoded array may look like this  [0, 0, 0, 1, 0], this states that the 4th element is the hot indicating element. This simply means that the image is from the 4th class. This samme applivcation is applied to this P&ID image project.

In [47]:
random_state = 42
# # performing one-hot encoding on our targets/labels 
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y_train_1cnn  = labelencoder.fit_transform(y_train_1cnn)
y_test_1cnn = labelencoder.fit_transform(y_test_1cnn)

y_train_2cnn  = labelencoder.fit_transform(y_train_2cnn)
y_test_2cnn = labelencoder.fit_transform(y_test_2cnn)

In [48]:
from keras.utils import np_utils
Y_train_categorical = np_utils.to_categorical(y_train_1cnn,65)
Y_test_categorical = np_utils.to_categorical(y_test_1cnn, 65)

print(len(Y_test_categorical))

730


In [49]:

Y_train_categorical_2 = np_utils.to_categorical(y_train_2cnn, 39)
Y_test_categorical_2 = np_utils.to_categorical(y_test_2cnn, 39)
print(Y_test_categorical_2.shape)
print(len(Y_test_categorical_2))

(730, 39)
730


In [ ]:
# labelencoder = LabelEncoder()
# y_test_2cnn  = labelencoder.fit_transform(y_test_2cnn)
# y_test_1cnn  = labelencoder.fit_transform(y_test_1cnn)
# print(y_test_1cnn)

In [ ]:
# random_state = 42
# labels_1 = pd.get_dummies(y_train_1cnn).values
# print(labels_1)

In [ ]:
# random_state = 42
# labels_2 = pd.get_dummies(y_train_2cnn).values
# print(labels_2)

In [ ]:
# labels_1_1 = pd.get_dummies(y_test_1cnn).values
# labels_2_2 = pd.get_dummies(y_test_2cnn).values
# print(labels_1_1)


Building our model by the use of Keras API which uses TensorFlow. 
For this section,I will import the Sequential Model from Keras and add Dense layers.
The softmax activation function is used in neural networks when building  a multi-class classifier. This  solves the problem  by assigning one instance to one class when the number of possible classes is greater than two.


In [179]:
# creating the model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.optimizers import SGD
random_state = 42
#model1 : Symbols_pixel_cd.csv
model = Sequential()
model.add(Conv2D(32,(3,3),activation='relu',kernel_initializer='uniform',input_shape=(100,100,1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(40, activation='relu'))
#sgd = SGD(lr=0.01, decay=0.00371, momentum=0.9, nesterov=False)
model.add(Dense(65, activation='softmax'))#The number of classes for this model is 65

#model_2 : Symbols_pixel.csv
model_2 = Sequential()
model_2.add(Conv2D(32,(3,3),activation='relu',kernel_initializer='uniform',input_shape=(100,100,1)))
model_2.add(Conv2D(32, (3, 3), activation='relu'))
model_2.add(MaxPooling2D(pool_size=(2,2)))
model_2.add(Flatten())
model_2.add(Dense(40, activation='relu'))
#sgd = SGD(lr=0.01, decay=0.00371, momentum=0.9, nesterov=False)
model_2.add(Dense(39, activation='softmax')) # The number of classes for this model is 39



Setting optimizer parameter to adam because it is Straightforward to implement,Computationally efficient and involves Little memory requirements.
settong loss parameter to categorical_crossentropy because this it is a multiclass problem.Also, tjhats tjhe reason the labels where converted to categorical format in previous chunks.

In [181]:
#model1 : Symbols_pixel_cd.csv
random_state = 42 
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

#model_2 : Symbols_pixel.csv
model_2.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])


In [175]:
#Fitting the model

random_state = 42
print("model_1 : Symbols_pixel_cd.csv")


print("/n")


model.fit(X_train_1cnn,Y_train_categorical, epochs=5)



model_1 : Symbols_pixel_cd.csv
/n
Epoch 1/5
1702/1702 [==============================] - 66s 39ms/step - loss: 2.8825 - accuracy: 0.3549
Epoch 2/5
1702/1702 [==============================] - 69s 41ms/step - loss: 0.9460 - accuracy: 0.7591
Epoch 3/5
1702/1702 [==============================] - 71s 42ms/step - loss: 0.3121 - accuracy: 0.9148
Epoch 4/5
1702/1702 [==============================] - 71s 42ms/step - loss: 0.1431 - accuracy: 0.9612
Epoch 5/5
1702/1702 [==============================] - 72s 42ms/step - loss: 0.1072 - accuracy: 0.9730


In [184]:
#Fitting the model
random_state = 42
print("model_2 : Symbols_pixel.csv")

print("/n")

model_2.fit(X_train_2cnn,Y_train_categorical_2 , epochs=5)


model_2 : Symbols_pixel.csv
/n
Epoch 1/5
1702/1702 [==============================] - 70s 41ms/step - loss: 0.5662 - accuracy: 0.8837
Epoch 2/5
1702/1702 [==============================] - 66s 39ms/step - loss: 0.1649 - accuracy: 0.9642
Epoch 3/5
1702/1702 [==============================] - 67s 40ms/step - loss: 0.0279 - accuracy: 0.9953
Epoch 4/5
1702/1702 [==============================] - 69s 40ms/step - loss: 0.0060 - accuracy: 0.9994
Epoch 5/5
1702/1702 [==============================] - 69s 40ms/step - loss: 0.0018 - accuracy: 1.0000


### Performance Evaluation Metrics

In [90]:
loss, accuracy = model.evaluate(X_test_1cnn,Y_test_categorical,verbose=0)
print('Loss: ', loss,'\nAcc: ', accuracy * 100)




Loss:  4.582426386010157 
Acc:  55.61643838882446


In [183]:
loss, accuracy = model_2.evaluate(X_test_2cnn,Y_test_categorical_2,verbose=0)
print('Loss: ', loss,'\nAcc: ', accuracy * 100)


Loss:  2.6319936458378623 
Acc:  59.86301302909851


In [162]:

predicted_labels = model.predict_classes(X_test_1cnn,batch_size=128,verbose=0)
#print(predicted_labels)
#convert target to single-digit ones,
rounded_labels=np.argmax(Y_test_categorical, axis=1)
cnn = classification_report(predicted_labels,rounded_labels)
print(cnn)

          

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00        18
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00       163
           6       0.00      0.00      0.00         6
           7       0.00      0.00      0.00        24
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         0
          17       0.00    

In [196]:

predicted_labels_2 = model.predict_classes(X_test_2cnn,batch_size=128,verbose=0)
#print(predicted_labels)
#convert target to single-digit ones,
rounded_labels_2=np.argmax(Y_test_categorical_2, axis=1)
cnn_2 = classification_report(predicted_labels_2,rounded_labels_2)
print(cnn_2)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         8
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         5
           6       0.00      0.00      0.00         0
           7       0.17      0.06      0.09        17
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00        17
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00        12
          16       0.00      0.00      0.00        15
          17       0.00    

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\hp\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### Summary of Results

#### RandomForest Results

In [207]:
pd.read_csv("RandomForest_Models.csv")


,Models_RF,Prescision,Recall,F1_Score,Accuracy
0,Symbols_pixel_cd,84%,84%,83%,83.70%
1,Symbols_pixel,92%,93%,91%,93%
2,Symbols_hog,93%,93%,93%,93.42%
3,Symbols_lbp,97%,97%,97%,93%
4,Symbols_hog_ros,100%,100%,100%,99.97%


#### Neural Network Results

In [30]:
pd.read_csv("NeuralNetwork_Models.csv")

,Models_NN,Prescision,Recall,F1_Score,Accuracy
0,Symbols_pixel_cd,77%,76%,76%,76.44%
1,Symbols_pixel,83%,81%,81%,81%
2,Symbols_hog,90%,90%,90%,89.72%
3,Symbols_lbp,54%,64%,56%,64%
4,Symbols_hog_ros,100%,100%,100%,99.73%


####  Convolutional Neural Network Results


In [197]:
pd.read_csv("CNN_Models.csv")

,Models_CNN,Prescision,Recall,F1_Score,Accuracy
0,Symbols_pixel_cd,8%,1%,1%,55.61%
1,Symbols_pixel,8%,3%,4%,60%


* A: Which is the best combination and why do you think so?
 As seen,from the result table section, the RandomForest  is best suitable for all the combination because it has an average accuracy across all models of 92.62,when compared to Neural Network and CNN which have 82.18% and 57.8.
Furthermore, in terms of precision which is calculated as the sum of True Positives(TP) across all classes and divided by the sum of true positives and False Positives(FP) across all classes,the RandomForest all has the highest average value of 93.2, when compared to  Neural Network and CNN which have 80.8% and 8%
Recall which is calculated as the sum of True Positives across all classes and divided by the sum of True Positives and False negatives.The classifier with e highest recall is still the  RandomForest with an avergae recall of 93.4 which tops the average racall of the Neural Network Classifier and CNN.The same applies to F1 score is directly prpoertional to the recall and precesion.Therefore a high recall and precsion results to a high F1_score.

Concisely, The RandomForest is best suitable for the combination as it preforms the best across all evaluation metrics used.Also, This classifier is moore computationally efficient when compared to the others used in this project.




* B: Which metric is more significant in this scenario?
Rating the most signficant metrics depends on the goal aor dataset.However, in this project, precsion, recision, recall and F1_Score are equally said to be significant for this scenario assuming we had a One-vs-All classifier which means the “micro” average or the “macro” average can be used and the performance can be calculated e.g., precision, from the individual TP,TN,FP, and FN of the the k-class model but in this case, the most signficant will be the classiffication accuarcy beacuse the labels in the datasets are all uniformly distributed.

* C: Is there any trade-off in using any combination in favour of others?
Naturally, models of the same dataset and set parameters, can perfrom differntly  depending on the classifier used. However, considering the results attained in this project, when two models of the same dataset and same set parameters are placed side by side from each classifer, the RandomForest classifier shows more accuarcy , precison,. recall and F1_score.Therefore, in this case, there is no trade-off which favours any another combination.


* D: How likely is to improve the obtained results provided that you have more resources/time?
More resources or data certainly improves the accuracy of the results in sampling to an extent,as the the Central Limit Theorem states, "The larger the size of the sample, the smaller the confidence band around the results".Howebver, at some point where adding an  infinite amount of data would not improve accuracy or results any further.This is point in known as the natural noise of the data.
In addition, by using different ML models, it is common that certain features of  the  data are spread to a given variance i.e. probabilistic distribution and it does not  matter how much training, test, validation, model complexity and bias-variance trade off considered in attempt to  improve the model it simply would not improve further.


